In [2]:
from cell_net import cellNetDataset
import numpy as np
import pandas as pd
import torch
import torch.nn as nn  
import pickle
from tqdm import tqdm
import networkx as nx
import pickle
import os.path
import os
import argparse
import shutil
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATv2Conv
###参数
# parser = argparse.ArgumentParser()
# parser.add_argument("-e", "--exp", help="file path of gene expression (tumor VS normal)")
# parser.add_argument("-g", "--graph", help="dir path of pyg data")
# parser.add_argument("-c", "--cell", help="file name of predicted cell (csv)")
# parser.add_argument("-n", "--net", help="dir path of sample enzyme net")
# parser.add_argument("-t", "--cores", help="multiprocess cores for creating pyg data")
# parser.add_argument("-m", "--model", help="file path of model .pt file")
# parser.add_argument("-d", "--datatype", help="test or val")
# parser.add_argument("-b", "--batchsize", help="batch size")
# parser.add_argument("-o", "--output", help="output file name")
# parser.add_argument("-a", "--gpu", help="cpu or gpu")
# args = parser.parse_args()
exp_file = "/home/data/sdb/wt/model_data/geo_chemo_diff_exp.csv"
pyg_dir = "/home/data/sdb/wt/model_data/tmp/geo_chemo/"
pre_file = "~/DeepMeta/data/geo_chemo_cells.csv"
net_dir = "/home/data/sdb/wt/model_data/geo_chemo_net/"
cores = 30
model_file = "/home/data/sdc/wt/model_data/new_model/enzyme_model_filterV2.pt"
# out_file = args.output
# pre_type = args.datatype
batch_size = 2
device = "cpu"
###create dir
pyg_dir_raw = pyg_dir + "/raw/"
pyg_dir_processed = pyg_dir + "/processed/"
if not os.path.exists(pyg_dir_raw):
    os.makedirs(pyg_dir_raw)
if not os.path.exists(pyg_dir_processed):
    os.makedirs(pyg_dir_processed)

train_genes = pd.read_csv("/home/wt/meta_target/data/train_genes.csv")
###mv pre_file to raw dir
if not os.path.exists(pyg_dir_raw + os.path.basename(pre_file)):
    shutil.copy(pre_file, pyg_dir_raw)

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATv2Conv(3247, 512, heads=3)
        self.conv2 = GATv2Conv(3 * 512, 512, heads=3)
        self.conv3 = GATv2Conv(3 * 512, 512, heads=3)
        self.lin1 = torch.nn.Linear(3 * 512 + 512, 1024)
        self.lin2 = torch.nn.Linear(1024, 512)
        self.lin3 = torch.nn.Linear(512, 1)
        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(7993, 4000),
            torch.nn.ReLU(),
            torch.nn.Linear(4000, 1500),
            torch.nn.ReLU(),
            torch.nn.Linear(1500, 512)
        )

    def forward(self, x, edge_index, exp):
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = torch.relu(self.conv3(x, edge_index))
        cell_end = self.encoder(exp)
        
        x = torch.cat((x, cell_end),1)
        x = torch.relu(self.lin1(x))
        x = torch.relu(self.lin2(x))
        x = self.lin3(x)
        return x
    
exp = pd.read_csv(exp_file)
cell_net = cellNetDataset(root = pyg_dir,
                          filename = os.path.basename(pre_file),
                          exp = exp,
                          data_type = "test", 
                          net_path = net_dir, 
                          cores = int(cores))
data_loader = DataLoader(cell_net, batch_size = batch_size, shuffle = False)

model = Net().to(device)
model.load_state_dict(torch.load(model_file, map_location=device))

<All keys matched successfully>

In [3]:
cell_net[0]

Data(edge_index=[2, 10295], nodename=[610], x=[610, 3247], exp=[610, 7993], cell='GSM1233067')

In [6]:
model.eval()
pre_type = "val"
for i, data in enumerate(tqdm(data_loader)):
    data = data.to(device)
    out_scale = model(data.x.float(), data.edge_index, data.exp.float())
    preds = np.rint(torch.sigmoid(out_scale).cpu().detach().numpy().flatten())
    preds_raw = torch.sigmoid(out_scale).cpu().detach().numpy().flatten()
    gene_name = np.array([i for batch in data.nodename for i in batch])
    if pre_type == "test":
        ys = data.y.cpu().detach().numpy()
        dt = {'preds':preds[data.y_index],'preds_raw':preds_raw[data.y_index],'gene_name':gene_name[data.y_index],"label":ys}
    else:
        dt = {'preds':preds,'preds_raw':preds_raw,'gene_name':gene_name}
    if i == 0:
        df = pd.DataFrame(dt)
        df["cell"] = data.cell[0]
    else:
        df1 = pd.DataFrame(dt)
        df1["cell"] = data.cell[0]
        df = pd.concat([df, df1])

100%|██████████| 725/725 [06:22<00:00,  1.89it/s]


In [7]:
df

,preds,preds_raw,gene_name,cell
0,0.0,0.075328,ENSG00000160282,GSM1233067
1,0.0,0.219928,ENSG00000107798,GSM1233067
2,0.0,0.285216,ENSG00000171298,GSM1233067
3,0.0,0.181617,ENSG00000002726,GSM1233067
4,0.0,0.017135,ENSG00000131471,GSM1233067
...,...,...,...,...
1215,0.0,0.045546,ENSG00000160191,GSM465240
1216,0.0,0.013865,ENSG00000172572,GSM465240
1217,0.0,0.000769,ENSG00000184588,GSM465240
1218,0.0,0.001571,ENSG00000186642,GSM465240


In [9]:
df.to_csv("/home/wt/DeepMeta/data/geo_test.csv")

In [ ]:
df = df[(df.gene_name.isin(train_genes.id))]
df.to_csv(out_file)